# Dataset creation for classification

In [3]:
#!pip install tensorflow
#!pip install seaborn

In [33]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import nltk
from utils_processor.processor import Processor
import logging
import os
import re
from tensorflow.keras.utils import to_categorical
import gensim
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers  # Añadimos esta importación
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers  # Añadimos esta importaciónimport tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K


In [2]:
processor_ = Processor()

In [3]:
import os
import re

# Directorio donde están los archivos .txt
data_dir = 'data/books/'

# Lista para almacenar los textos y autores
texts = []
authors = []

# Leer todos los archivos .txt del directorio
for filename in os.listdir(data_dir):
    if filename.endswith('.txt'):

        with open(os.path.join(data_dir, filename), 'r', encoding='utf-8') as f:
            text = f.read()
            texts.append(text)  # Almacenar el texto
            print(f"Procesando archivo: {filename}, lineas: ", len(text))
            
            # Buscar el nombre del autor
            author_match = re.search(r'Author:\s*(.+)', text)
            if author_match:
                author_name = author_match.group(1).strip()
                authors.append(author_name.lower())  # Almacenar el autor
            else:
                authors.append("Autor no encontrado")  # En caso de no encontrarlo
            
            # Mostrar un fragmento del texto (opcional)
            #print(text[:2500])

# Mostrar los autores encontrados
print("Lista de autores en orden:")
for i, author in enumerate(authors):
    print(f"{i+1}. {author}")

# Ahora tienes dos listas: 'texts' con los textos y 'authors' con los autores en el mismo orden
authors

Procesando archivo: ATaleofTwoCities_Dickens.txt, lineas:  776878
Procesando archivo: BleakHouse_Dickens.txt, lineas:  1958792
Procesando archivo: CountofMonteCristo_Dumas.txt, lineas:  2646641
Procesando archivo: CrimeAndPunishment_dostoyevski.txt, lineas:  1154409
Procesando archivo: OliverTwist_Dickens.txt, lineas:  912421
Procesando archivo: TheGambler_dostoyevski.txt, lineas:  350954
Procesando archivo: TheIdiot_dostoyevski.txt, lineas:  1366983
Procesando archivo: TheThreeMusketeers_Dumas.txt, lineas:  1317339
Procesando archivo: TwentyYearsAfter_Dumas.txt, lineas:  1387344
Lista de autores en orden:
1. charles dickens
2. charles dickens
3. alexandre dumas
4. fyodor dostoyevsky
5. charles dickens
6. fyodor dostoyevsky
7. fyodor dostoyevsky
8. alexandre dumas
9. alexandre dumas


['charles dickens',
 'charles dickens',
 'alexandre dumas',
 'fyodor dostoyevsky',
 'charles dickens',
 'fyodor dostoyevsky',
 'fyodor dostoyevsky',
 'alexandre dumas',
 'alexandre dumas']

In [4]:
def process_all_texts(processor = Processor(), texts: list = []):
    """
    Processes a list of texts and logs progress for each one, using the Processor class.
    
    Args:
        processor (Processor): An instance of the Processor class.
        texts (list): A list of text strings to process.
    
    Returns:
        list: A list of processed texts.
    """
    total = len(texts)
    processed_texts = []
    
    for index, text in enumerate(texts):
        processed_text = processor.preprocessing_pipeline_as_chunks(text, index, total)
        processed_texts.append(processed_text)  # Guardamos el texto procesado como lista de tokens
    
    return processed_texts

In [5]:
# Procesar todos los textos con el sistema de logging
processed_texts = process_all_texts(processor_, texts)

In [6]:
text_chunks = []
chunk_authors = []

for i, text_list in enumerate(processed_texts):
    author = authors[i]
    for chunk in text_list:
        text_chunks.append(chunk)  # Agregar cada chunk de texto
        chunk_authors.append(author)  # Agregar el autor correspondiente

# Crear un DataFrame con las listas
df_chunks = pd.DataFrame({
    'text_chunk': text_chunks,
    'author': chunk_authors
})


In [7]:
df_chunks

,text_chunk,author
0,tale num citi tale num citi stori french revol...,charles dickens
1,chapter xv knit chapter xvi still knit chapter...,charles dickens
2,life chapter period best time worst time age w...,charles dickens
3,king larg jaw queen plain face throne england ...,charles dickens
4,london westminst even cocklan ghost laid round...,charles dickens
...,...,...
16890,critic reach project gutenberg goal ensur proj...,alexandre dumas
16891,num contribut project gutenberg literari archi...,alexandre dumas
16892,num num particular import maintain tax exempt ...,alexandre dumas
16893,us offer donat intern donat grate accept make ...,alexandre dumas


1. Prepare the text data

We already have the processed texts stored in a list called processed_texts. Each element in this list represents the chunks of text (after splitting) for a particular book.
Each entry in processed_texts is a list where each element is a chunk of text for that book, processed based on the method we implemented for splitting into chunks of 150 words with a 25-word overlap

2. Prepare the author labels

We have an authors list that stores the corresponding author for each book in processed_texts. Each author appears multiple times if they have multiple books in the dataset. For example:
python

authors = ['dostoyevski', 'poe', 'dostoyevski', 'dostoyevski', 'well', 'poe', 'poe', 'well', 'well']

3. Create the DataFrame structure

For each processed book (i.e., processed_texts[i]), we know that all the chunks of that book correspond to a specific author. So we can assign the same author to all the chunks in that list.
We will loop over each entry in processed_texts and for each chunk, add it to a DataFrame, along with the corresponding author.

4. Steps to build the DataFrame

* Initialize lists for the DataFrame: We will initialize two lists: one for text chunks and one for authors.
* Iterate over processed_texts: For each entry in processed_texts, we extract the list of chunks and the corresponding author.
* Add chunks and authors to the lists: For each chunk in the list of text chunks, we append it to the "text_chunk" list and the corresponding author to the "author" list.
* Create the DataFrame: Once the lists are filled, we create a pandas DataFrame with two columns: "text_chunk" and "author".

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming df_chunks is the dataframe with the columns ['text_chunk', 'author']

# Step 1: Split the dataset into 70% training and 30% test
train_df, test_df = train_test_split(df_chunks, test_size=0.30, stratify=df_chunks['author'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df['author'], random_state=42)


In [9]:
train_df

,text_chunk,author
10344,elbow beg pardon said dodger look air abstract...,charles dickens
2552,pursu say emphat william guppi drop mr guppi a...,charles dickens
13609,secret use know anyth said young woman instinc...,alexandre dumas
3369,look keep secret condescens present visit feel...,charles dickens
9896,jew sooner alon counten resum former express a...,charles dickens
...,...,...
5799,better inform know owner hors shut cri pit cho...,alexandre dumas
8183,began count third excus would say fanci made m...,fyodor dostoyevsky
13251,take away commiss give mademoisell de chemerau...,alexandre dumas
4851,must curios natur island mass rock contain acr...,alexandre dumas


In [10]:
val_df

,text_chunk,author
5881,deceiv play joke excel read ah true said mont ...,alexandre dumas
882,heel head wish wos still say prewar sir let bo...,charles dickens
16471,would choos num atho artagnan said noth silenc...,alexandre dumas
3294,littl earlier morn keep account attend houseke...,charles dickens
13959,shall get back upon lackey hors _pardieu_ anyb...,alexandre dumas
...,...,...
8226,heart long anoth father polya papa fear angri ...,fyodor dostoyevsky
11409,note often grow paler take princ took note fer...,fyodor dostoyevsky
7376,room first floor room whitewash custom prison ...,alexandre dumas
1898,time littl woman ad rub head signific settl ye...,charles dickens


In [11]:
test_df

,text_chunk,author
12339,quit sure reach culmin point happi num day saw...,fyodor dostoyevsky
4185,spain itali mercédè father could join fear liv...,alexandre dumas
15378,found pale fatigu inquir whether ill fact said...,alexandre dumas
9926,empti comfort said mrs corney much inde said b...,charles dickens
6920,crush singl touch word breath yes self thought...,alexandre dumas
...,...,...
2448,poor dear girl found much admir good disposit ...,charles dickens
14473,dispos convers reclin corner carriag num pass ...,alexandre dumas
14223,smile indic knew stori well wish relat recomme...,alexandre dumas
16470,shall begin portho arami drew back disappoint ...,alexandre dumas


In [12]:

def summary_by_author(train_df, validation_df, test_df):
    """
    Generates a summary table showing the number of samples per author for the training, validation, and testing sets.
    
    Args:
        train_df (pd.DataFrame): Training DataFrame.
        validation_df (pd.DataFrame): Validation DataFrame.
        test_df (pd.DataFrame): Testing DataFrame.
        
    Returns:
        pd.DataFrame: A summary DataFrame.
    """
    
    summary_data = {
        'Author': train_df['author'].unique(),
        'Train': train_df['author'].value_counts(),
        'Validation': validation_df['author'].value_counts(),
        'Test': test_df['author'].value_counts()
    }
    
    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.fillna(0)  # Replace NaN with 0 if no samples exist for some authors
    
    return summary_df

In [13]:
summary_by_author(train_df, val_df, test_df)

,Author,Train,Validation,Test
author,,,,
alexandre dumas,charles dickens,4744,527,2260
charles dickens,alexandre dumas,3307,368,1575
fyodor dostoyevsky,fyodor dostoyevsky,2592,288,1234


Feed Forward

## Word2Vec large embedding

In [18]:
from tensorflow.keras.utils import to_categorical
import gensim

In [19]:
# Cargar el modelo de GoogleNews
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('data/largest_embedding/GoogleNews-vectors-negative300.bin', binary=True)

In [26]:
def prepare_data(df, processor_, word2vec_model):
    X = []
    print("Procesando textos...")
    
    # Definir índices especiales
    PAD_IDX = 0
    UNK_IDX = 1
    
    # Primero, encontrar todas las palabras únicas en nuestros datos
    unique_words = set()
    for text in df['text_chunk']:
        processed_tokens = processor_.preprocessing_pipeline(text)
        unique_words.update(processed_tokens)
    
    # Crear un vocabulario limitado solo a las palabras que aparecen en nuestros datos
    # y que también están en word2vec
    valid_words = {word for word in unique_words if word in word2vec_model.key_to_index}
    
    # Crear un nuevo mapeo de índices
    word_to_idx = {word: (idx + 2) for idx, word in enumerate(valid_words)}
    vocab_size = len(word_to_idx) + 2  # +2 para PAD y UNK
    
    print(f"Palabras únicas en datos: {len(unique_words)}")
    print(f"Palabras encontradas en word2vec: {len(valid_words)}")
    print(f"Tamaño final del vocabulario (incluyendo PAD y UNK): {vocab_size}")
    
    # Procesar los textos con el nuevo vocabulario
    for text in df['text_chunk']:
        processed_tokens = processor_.preprocessing_pipeline(text)
        indices = [word_to_idx.get(token, UNK_IDX) for token in processed_tokens]
        X.append(indices)
    
    return X, vocab_size, word_to_idx

def create_embedding_matrix(word2vec_model, word_to_idx, embedding_size):
    vocab_size = len(word_to_idx) + 2  # +2 para PAD y UNK
    embedding_matrix = np.zeros((vocab_size, embedding_size))
    
    # Vector para PAD (todos ceros)
    embedding_matrix[0] = np.zeros(embedding_size)
    
    # Vector para UNK (promedio de todos los vectores de palabras válidas)
    valid_vectors = [word2vec_model[word] for word in word_to_idx.keys()]
    embedding_matrix[1] = np.mean(valid_vectors, axis=0)
    
    # Llenar vectores para palabras del vocabulario
    for word, idx in word_to_idx.items():
        embedding_matrix[idx] = word2vec_model[word]
    
    return embedding_matrix

# Preparar los datos
print("Preparando datos de entrenamiento...")
X_train, vocab_size, word_to_idx = prepare_data(train_df, processor_, word2vec_model)
print("\nPreparando datos de validación...")
X_val, _, _ = prepare_data(val_df, processor_, word2vec_model)
print("\nPreparando datos de prueba...")
X_test, _, _ = prepare_data(test_df, processor_, word2vec_model)

# Padding
print("\nAplicando padding...")
max_length = max(
    max(len(x) for x in X_train),
    max(len(x) for x in X_val),
    max(len(x) for x in X_test)
)
print(f"Longitud máxima de secuencia: {max_length}")

X_train_padded = pad_sequences(X_train, maxlen=max_length, padding='post')
X_val_padded = pad_sequences(X_val, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test, maxlen=max_length, padding='post')

label_encoder = LabelEncoder()
label_encoder.fit(train_df['author'])

y_train = label_encoder.transform(train_df['author'])
y_val = label_encoder.transform(val_df['author'])
y_test = label_encoder.transform(test_df['author'])

classes = np.unique(y_train)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)
y_test_cat = to_categorical(y_test)


# Crear matriz de embeddings
print("\nCreando matriz de embeddings...")
embedding_matrix = create_embedding_matrix(word2vec_model, word_to_idx, embedding_size=300)

# Crear y entrenar el modelo

Preparando datos de entrenamiento...
Procesando textos...
Palabras únicas en datos: 22297
Palabras encontradas en word2vec: 8416
Tamaño final del vocabulario (incluyendo PAD y UNK): 8418

Preparando datos de validación...
Procesando textos...
Palabras únicas en datos: 8788
Palabras encontradas en word2vec: 4754
Tamaño final del vocabulario (incluyendo PAD y UNK): 4756

Preparando datos de prueba...
Procesando textos...
Palabras únicas en datos: 16474
Palabras encontradas en word2vec: 7121
Tamaño final del vocabulario (incluyendo PAD y UNK): 7123

Aplicando padding...
Longitud máxima de secuencia: 121

Creando matriz de embeddings...


In [27]:
print("Forma y tipo de las etiquetas:")
print("y_train shape:", y_train_cat.shape)
print("Valores únicos en y_val:", y_val_cat)
print("Valores únicos en y_test:", np.unique(y_test_cat))
print("Valores únicos en y_train:", np.unique(y_train_cat))
print("Tipo de datos de y_train:", y_train_cat.dtype)

Forma y tipo de las etiquetas:
y_train shape: (10643, 3)
Valores únicos en y_val: [[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
Valores únicos en y_test: [0. 1.]
Valores únicos en y_train: [0. 1.]
Tipo de datos de y_train: float64


In [28]:
checkpoint_path = "best_model.h5"  # Donde se guardará el mejor modelo


In [34]:
print("\nCreando modelo...")
model = create_latest_model(
    vocab_size=vocab_size, 
    embedding_dim=300,  # Ya que usas GoogleNews que tiene 300 dimensiones
    embedding_matrix=embedding_matrix,
    max_length=max_length,
    n_classes=3  # Para las 3 categorías que mencionas
)

# Verificación final
print("\nVerificación final de índices:")
print(f"Rango válido: [0, {vocab_size})")
print(f"Máximo en train: {X_train_padded.max()}")
print(f"Máximo en val: {X_val_padded.max()}")
print(f"Máximo en test: {X_test_padded.max()}")

# Entrenar el modelo
history = model.fit(
    X_train_padded, 
    y_train_cat,
    validation_data=(X_val_padded, y_val_cat),
    epochs=30,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=create_latest_callbacks(checkpoint_path),
    verbose=1
)


Creando modelo...


AttributeError: module 'keras.src.activations' has no attribute 'get'

In [140]:
test_loss, test_accuracy = simplified_model.evaluate(X_test_padded, y_test)
print(f"\nTest accuracy: {test_accuracy:.4f}")


159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2515 - loss: 1.0989

Test accuracy: 0.2436


In [141]:
# Generate classification report
y_pred = simplified_model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))

159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step

Classification Report:
                    precision    recall  f1-score   support

   alexandre dumas       1.00      0.00      0.00      2260
   charles dickens       0.00      0.00      0.00      1575
fyodor dostoyevsky       0.24      1.00      0.39      1234

          accuracy                           0.24      5069
         macro avg       0.41      0.33      0.13      5069
      weighted avg       0.51      0.24      0.10      5069



C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classi